In [ ]:
!pip install awswrangler

In [5]:
!pip install fastparquet

Keyring is skipped due to an exception: 'keyring.backends'


In [6]:
import boto3
import pandas as pd
import numpy as np
import re
from sagemaker import get_execution_role
import awswrangler as wr
import os, sys
role = get_execution_role()
s3 = boto3.resource('s3')

In [7]:
items = wr.s3.read_parquet(path="s3://icusics-db/d_variables/d_variables.parquet")


In [10]:
valuestring= items.choicestringvalue.value_counts().reset_index()

items_related_to_crrt= 'Dialysate|HDFVVC|HDVVC|HFVVC|hemof|CVVH|CVVHDF|CVVHD|dyal|crrt|rrt|diàlisi|diálisis|dialysis|dial|teràpia substitutiva renal|renal|dialisi|fluxe'
items_to_exclude = 'SOFA|apache|saps'

items['crrt']= items[['name', 'abbreviation','description','choicestringvalue']].apply(lambda x: x.str.contains(items_related_to_crrt, case=False)).any(axis=1).astype(int)
items_crrt = items[items.crrt==1]
items_crrt=items_crrt.drop_duplicates(subset=['name', 'abbreviation','description'])

items_crrt['crrt']= items_crrt[['name', 'abbreviation','description','choicestringvalue']].apply(lambda x: x.str.contains(items_to_exclude, case=False)).any(axis=1).astype(int)
items_crrt = items_crrt[items_crrt.crrt==0]

# manually removed  

to_remove = [5015003961,3014000040,6015004233,6015004051,1015003669,1015003670,
             6015003952,5015004023,6000009255,6015001063,6015001673,6015002807,
             3015003687,3015003163,3015003474,1015002807,1015003663,1015002807,
             6015002807,5015003793,1015002872,5015004024,4015003815]

items_crrt=items_crrt[~items_crrt.a_variableid.isin(to_remove)]

items_crrt=items_crrt.sort_values(by='hospital_coded')

items_crrt= items_crrt.rename(columns={'crrt':'is_crrt'})

#items_crrt.to_excel("items_for_crrt.xlsx")

In [11]:
items_crrt

,hospital_coded,table,a_variableid,choicecode,choicestringvalue,name,abbreviation,description,is_crrt
74,1,derived_numeric,1030005640,NaN,<NA>,Cum ultrafiltrate,Ultrafiltr/c,Cumulative value of ultrafiltrate (OUTeffluen/...,0
4157,1,observed_numeric,1015001590,NaN,<NA>,Balanç HF (horari),Balanç HF,M6 T. CONTÍNUES DEPURACIÓ EXTRARENAL,0
4156,1,observed_numeric,1015001587,NaN,<NA>,Heparina a l'Hemofiltre,Heparina HF,M6 T. CONTÍNUES DEPURACIÓ EXTRARENAL,0
4155,1,observed_numeric,1015001580,NaN,<NA>,Fluxe Diàlisi pautat,Qd,M6 T. CONTÍNUES DEPURACIÓ EXTRARENAL,0
4154,1,observed_numeric,1015001579,NaN,<NA>,Fluxe Sang pautada,Qs,M6 T. CONTÍNUES DEPURACIÓ EXTRARENAL,0
...,...,...,...,...,...,...,...,...,...
26726,6,monitored_numeric,6000005649,NaN,<NA>,Qheparina_Fluxe Bomba heparina,Qheparina,Monitored variable 14 10000 Ui HepNa + 50 mL S...,0
26756,6,monitored_numeric,6000009222,NaN,<NA>,Qdiàlisi_Flujo de Dialisis,Qdial,Fresenius Multifiltrate,0
27016,6,observed_categoric,6010002407,0.0,No,Tècniques TCRR,Tècnica TCRR,7. Escala NEMS ( Tècniques de diàlisi: TOTES),0
26689,6,monitored_numeric,6000001404,NaN,<NA>,Qr Post-replacement input,Qr PostF (m),Post-filter replacement input from RRT device....,0
